In [3]:
import sys
sys.path.append("/Users/utilizador/Documents/GitHub/si/src")
import numpy as np
from si.data.dataset import Dataset

A classe LassoRegression implementa a regressão Lasso, um método de regressão linear regularizado. O principal objetivo do Lasso é ajustar um modelo linear aos dados enquanto aplica uma penalização baseada na soma dos valores absolutos dos coeficientes. Isso resulta em um modelo esparso, onde alguns coeficientes podem ser exatamente zero, o que é útil para seleção de variáveis.

Descida por Coordenadas:
O Lasso é resolvido usando um método chamado coordinate descent.
Em cada iteração, otimiza-se um coeficiente por vez, mantendo os outros fixos.

Resumo da Lógica
__init__: Configura os parâmetros do modelo.

_soft_threshold: Aplica o operador de soft-thresholding para ajustar coeficientes.

fit:

- Usa descida por coordenadas para ajustar os coeficientes.

- Atualiza cada coeficiente usando resíduos e normalização.

- Para as iterações se a mudança nos coeficientes for menor que a tolerância.

- predict: Calcula as previsões baseadas nas features e nos coeficientes ajustados.

- score: Avalia o desempenho do modelo com o MSE.

Vantagens e Limitações

Vantagens:

Esparsidade: Promove coeficientes exatamente zero, útil para seleção de variáveis.
Regularização: Ajuda a evitar overfitting.

Limitações:

Complexidade Computacional: Pode ser lento em datasets muito grandes.
Convergência: Requer boa inicialização e escolha adequada de λ.

In [36]:
class LassoRegression:
    def __init__(self, l1_penalty=1.0, max_iter=1000, tolerance=1e-4):
        """
        Lasso Regression implementation using coordinate descent.

        Parameters
        ----------
        l1_penalty : float
            The regularization parameter (lambda).
        max_iter : int
            Maximum number of iterations for optimization.
        tolerance : float
            Convergence tolerance for stopping criteria.
        """
        self.l1_penalty = l1_penalty
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.coef_ = None
        self.intercept_ = None

    def _soft_threshold(self, rho, lam):
        """
        Apply the soft-thresholding operator for Lasso.

        Parameters
        ----------
        rho : float
            The correlation between feature and residual.
        lam : float
            The regularization parameter (lambda).

        Returns
        -------
        float
            The updated coefficient value after soft-thresholding.
        """
        if rho < -lam:  # Se a correlação for menor que -lambda, reduz rho
            return rho + lam
        elif rho > lam:  # Se a correlação for maior que lambda, reduz rho
            return rho - lam
        else:   # Se rho estiver entre -lambda e lambda, o coeficiente é zerado
            return 0

    def fit(self, dataset):
        """
        Fit the Lasso Regression model.

        Parameters
        ----------
        dataset : Dataset
            A Dataset object containing features (X) and labels (y).
        """
        X, y = dataset.X, dataset.y
        n_samples, n_features = X.shape
        self.coef_ = np.zeros(n_features)  #Inicializa os coeficientes com zeros
        self.intercept_ = 0

        for iteration in range(self.max_iter):  # Itera até atingir o número máximo ou convergir
            coef_prev = self.coef_.copy()   # Copia os coeficientes para verificar convergência

            
            self.intercept_ = np.mean(y - np.dot(X, self.coef_))  # Atualiza o intercepto como a média dos resíduos

            for j in range(n_features):   # Atualiza cada coeficiente individualmente
                
                residual = y - (np.dot(X, self.coef_) + self.intercept_) # Calcula os resíduo
                rho = np.dot(X[:, j], residual + self.coef_[j] * X[:, j]) # Correlação com o resíduo
                self.coef_[j] = self._soft_threshold(rho, self.l1_penalty) / np.sum(X[:, j] ** 2) # Aplica soft-thresholding e normaliza pelo quadrado da feature

            # Verifica a convergência
            if np.sum(np.abs(self.coef_ - coef_prev)) < self.tolerance:
                break # Vai interromper se a mudança nos coeficientes for menor que a tolerância

    def predict(self, dataset):
        """
        Predict target values using the fitted Lasso model.

        Parameters
        ----------
        dataset : Dataset
            A Dataset object containing features (X).

        Returns
        -------
        ndarray of shape (n_samples,)
            The predicted target values.
        """
        X = dataset.X
        return np.dot(X, self.coef_) + self.intercept_

    def score(self, dataset):
        """
        Calculate the Mean Squared Error (MSE) of the predictions.

        Parameters
        ----------
        dataset : Dataset
            A Dataset object containing features (X) and labels (y).

        Returns
        -------
        float
            The Mean Squared Error of the predictions.
        """
        X, y = dataset.X, dataset.y
        y_pred = self.predict(dataset)
        return np.mean((y - y_pred) ** 2) # Calcula o MSE (não estava a coneguir usar a função dada na aula)



### Comparação do meu modelo com o do sekit-learn para ver se está a funcionar mais ou menos (podia ser pior...)

In [38]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

X_train = np.array([[1, 2], [2, 4], [3, 6], [4, 8], [5, 10]])
y_train = np.array([1, 2, 3, 4, 5])

# A minha 
dataset = Dataset(X_train, y_train)
model = LassoRegression(l1_penalty=1)
model.fit(dataset)
y_pred_custom = model.predict(dataset)
mse_custom = mean_squared_error(y_train, y_pred_custom)

# Modelo do scikit-learn
lasso = Lasso(alpha=1, fit_intercept=True, max_iter=1000)
lasso.fit(X_train, y_train)
y_pred_sklearn = lasso.predict(X_train)
mse_sklearn = mean_squared_error(y_train, y_pred_sklearn)

print("MSE - Custom:", mse_custom)
print("MSE - scikit-learn:", mse_sklearn)


MSE - Custom: 0.005000000109858807
MSE - scikit-learn: 0.125
